In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from irt.evaluation import eval_utils
from irt.data import data_loader
import matplotlib.pyplot as plt
import time
import girth
# from girth import rasch_conditional
from irt.data.rasch import generate_data
from irt.algorithms.spectral_estimator import spectral_estimate, construct_markov_chain, construct_markov_chain_accelerated
from irt.algorithms import conditional_mle
from irt.algorithms import rasch_mml
from irt.evaluation.eval_utils import log_likelihood_heldout, bayesian_auc, pairwise_disagreement_error, top_k_accuracy
import warnings
warnings.filterwarnings("ignore")

In [3]:
# A = data_loader.lsat()
# A_lsat = data_loader.lsat()
# A_ml_100k, ml_100k_ratings = data_loader.ml_100k(cutoff=cutoff)
# A_hetrec_2k, hetrec_2k_ratings  = data_loader.hetrec_2k(cutoff=50)
# A_ml_1m, ml_1m_ratings = data_loader.ml_1m(cutoff=cutoff, top_k_cutoff=200)
# A_bx, bx_ratings = data_loader.bx_book(cutoff=50, top_k_cutoff=200)
A_genome, genome_ratings = data_loader.book_genome(cutoff=200, top_k_cutoff=500)
# A_ml_20m, ml_20m_ratings = data_loader.ml_20m(cutoff=100, top_k_cutoff=200)

In [4]:
# A = A_ml_100k
# ratings = ml_100k_ratings
A = A_genome
ratings = genome_ratings

sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
true_rank = [item for (item, _, num_ratings) in sorted_ratings] # Sort from most popular items
ratings_dict = dict([(item, (rating, num_ratings)) for item, rating, num_ratings in ratings])

In [5]:
K = 200
print(f"Data has shape {A.shape}")
all_num_ratings = [num_ratings for (_, _, num_ratings) in sorted_ratings][:K]
print(f"mean num ratings in top {K}:", np.mean(all_num_ratings))
print(f"max num ratings in top {K}:", np.max(all_num_ratings))
print(f"median num ratings in top {K}:", np.median(all_num_ratings))
print(f"min num ratings in top {K}:", np.min(all_num_ratings))

Data has shape (7238, 350332)
mean num ratings in top 200: 1830.7
max num ratings in top 200: 12847
median num ratings in top 200: 1044.5
min num ratings in top 200: 505


In [6]:
est_ase = est_ase = spectral_estimate(A, lambd=1., regularization="uniform")
est_mmle = rasch_mml.rasch_mml(A, return_beta=True)

In [7]:
rank_ase = np.argsort(est_ase)[::-1]
rank_mmle = np.argsort(est_mmle)[::-1]

In [10]:
K = 200
print(f"ASE {len(np.intersect1d(true_rank[:K], rank_ase[:K]))}")
print(f"MMLE {len(np.intersect1d(true_rank[:K], rank_mmle[:K]))}")

# for item in true_rank[:K]:
#     if item not in rank_ase[:K]:
#         print(f"{item}, {ratings_dict[item]}")


ASE 113
MMLE 86
